In [2]:
import os

In [3]:
%pwd

'd:\\ML_Pro\\Chicken-Disease-Classification-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ML_Pro\\Chicken-Disease-Classification-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list 
    params_learning_rate: float

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf
from tensorflow import keras


In [8]:
from pathlib import Path
import os
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.entity.config_entity import (
    PrepareCallbacksConfig,
    TrainingConfig
)


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # ✅ Handle both script (.py) and Jupyter notebook
        if "__file__" in globals():
            self.root_dir = Path(__file__).resolve().parent.parent.parent
        else:
            self.root_dir = Path(os.getcwd()).resolve()

        # Make artifacts_root absolute
        self.config.artifacts_root = self.root_dir / self.config.artifacts_root
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        root_dir = self.root_dir / config.root_dir
        tensorboard_root_log_dir = self.root_dir / config.tensorboard_root_log_dir
        checkpoint_model_filepath = self.root_dir / config.checkpoint_model_filepath

        # Ensure dirs exist
        model_ckpt_dir = checkpoint_model_filepath.parent
        create_directories([model_ckpt_dir, tensorboard_root_log_dir])

        return PrepareCallbacksConfig(
            root_dir=root_dir,
            tensorboard_root_log_dir=tensorboard_root_log_dir,
            checkpoint_model_filepath=checkpoint_model_filepath
        )

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        root_dir = self.root_dir / training.root_dir
        trained_model_path = self.root_dir / training.trained_model_path
        updated_base_model_path = self.root_dir / prepare_base_model.updated_base_model_path

        # Training data absolute path
        training_data = self.root_dir / self.config.data_ingestion.unzip_dir / "Chicken-fecal-images"

        create_directories([root_dir])

        return TrainingConfig(
            root_dir=root_dir,
            trained_model_path=trained_model_path,
            updated_base_model_path=updated_base_model_path,
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )


In [9]:
import time

In [10]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [11]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.config.run_functions_eagerly(True)\
# Verify configuration
print(f"TensorFlow version: {tf.__version__}")
print(f"Eager execution enabled: {tf.executing_eagerly()}")

# Optional: Set memory growth for GPU (if using GPU)
try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) available: {len(gpus)}")
    else:
        print("No GPU available, using CPU")
except Exception as e:
    print(f"GPU configuration warning: {e}")

TensorFlow version: 2.20.0
Eager execution enabled: True
No GPU available, using CPU


In [12]:
import os
print("[DEBUG] Current Working Directory:", os.getcwd())


[DEBUG] Current Working Directory: d:\ML_Pro\Chicken-Disease-Classification-Project


In [13]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        print("[DEBUG] Trying to load model from:", self.config.updated_base_model_path)
        print("[DEBUG] Exists:", Path(self.config.updated_base_model_path).exists())


        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(

            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs

            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        path = Path(path)
        if path.suffix not in [".h5", ".keras"]:
        # default to new format
            path = path.with_suffix(".keras")
        model.save(path)
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [14]:
# ❌ Wrong
# from keras.preprocessing.image import ImageDataGenerator

# ✅ Correct
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping



In [15]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

[2025-09-08 12:40:09,194: INFO: common: Attempting to load YAML from D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml]
[2025-09-08 12:40:09,198: INFO: common: YAML file D:\ML_Pro\Chicken-Disease-Classification-Project\config\config.yaml loaded successfully]
[2025-09-08 12:40:09,200: INFO: common: Attempting to load YAML from D:\ML_Pro\Chicken-Disease-Classification-Project\params.yaml]
[2025-09-08 12:40:09,202: INFO: common: YAML file D:\ML_Pro\Chicken-Disease-Classification-Project\params.yaml loaded successfully]
[2025-09-08 12:40:09,204: INFO: common: created directory at: D:\ML_Pro\Chicken-Disease-Classification-Project\artifacts]
[2025-09-08 12:40:09,205: INFO: common: created directory at: D:\ML_Pro\Chicken-Disease-Classification-Project\artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-08 12:40:09,206: INFO: common: created directory at: D:\ML_Pro\Chicken-Disease-Classification-Project\artifacts\prepare_callbacks\tensorboard_root_log_dir]
[2025-09-08 12:40:

c:\Users\kunal\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\kunal\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6482 - loss: 0.7618

c:\Users\kunal\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-09-08 12:42:09,207: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 118s 6s/step - accuracy: 0.7736 - loss: 0.4843 - val_accuracy: 0.9375 - val_loss: 0.2680
[2025-09-08 12:42:09,366: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
